# MAE6592 Python Assignment: Part II

By Intelligent System Lab, University of Virginia


In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

This script imports three popular packages in python
1. numpy:  used to do scientific calculations
2. pandas: used to serve as database
3. pyplot: used to make charts

In [19]:
class serial_line:
    def __init__(self,cycle_time,buffer_capacity,buffer_initial=-1):
        """
        Arguments:
            cycle_time: the cycle time of each machine in minute, list or numpy array
            buffer_capacity: the maximum capacity of each buffer, list or numpy array
            buffer_initial: the initial buffer level of each buffer, zeros by default, list or numpy array
                            
        """
        self.M=len(cycle_time)                #length of the serial line
        self.ct=cycle_time                    #cycle time
        self.bc=buffer_capacity               #buffer capacity
        self.slowest=np.argmax(self.ct)       #find the slowest machine
        
        if buffer_initial==-1:
            self.bi=np.zeros(self.M-1)        #if initial buffer levels are not specified, start from zero
        else:
            self.bi=np.array(buffer_initial)  #initial buffer level
        
        """calculate buffer boundary"""
        self.beta=np.zeros((self.M,self.M))
        for i in range(self.M):
            for j in range(self.M):
                self.beta[i,j]=(sum(self.bc[i:j])-sum(self.bi[i:j]))\
                *(i<j)+sum(self.bi[j:i])*(i>j)

    def run(self,T,downtime,decimal=0):
        """
        Arguments:
            T: the total simulation time length in minute, int
            downtime: the downtime/ow you want to insert, tuple or list
                      ([[1st_timeofow,1st_durationofow],[2nd_timeofow,2nd_durationofow]]     #machine 1
                      [],                                                                    #machine 2, blank list if no ow inserted
                      ...,                                                                   #...
                      [[1st_timeofow,1st_durationofow]])                                     #machine M
            decimal: the decimals in the simulation time horizon, zero by default, int
        Return:
            stepwise buffer levels
            stepwise production counts
        """
        tpoints=T*(10**decimal)+1

        b=np.zeros((self.M-1,tpoints))         #buffer level
        b[:,0]=self.bi                         #initial buffer level
        pc=np.zeros((self.M,tpoints))          #production counts
        
        self.w=np.zeros((self.M,tpoints))      #generate downtime list according to downtime input
        for i in range(self.M):
            for j in downtime[i]:
                self.w[i,np.int(np.round(j[0]*(10**decimal))):\
                       np.int(np.round((j[0]+j[1])*(10**decimal)))]=1         #w=1 if the machine is down at the time step
        
        for t in range(1,tpoints):
            pc[:,t]=pc[:,t-1]+(1-self.w[:,t-1])/self.ct/(10**decimal)
            for i in range(self.M):
                temp=self.beta[i,:]+pc[:,t]
                pc[i,t]=np.min(temp)
            b[:,t]=pc[:self.M-1,t]-pc[1:,t]+self.bi                          #calculate buffer levels
        
        time=np.round(np.linspace(0,T,tpoints),decimals=decimal)             #generate time steps
        self.b=pd.DataFrame(b.T,
                            index=time,
                            columns=["B"+str(x) for x in range(2,self.M+1)]) #store results to pandas dataframe
        self.b=self.b.round(np.max((6,decimal)))                             #round results to remove numerical errors
        
        self.pc=pd.DataFrame(pc.T,
                             index=time,
                             columns=["M"+str(x) for x in range(1,self.M+1)])
        self.pc=self.pc.round(np.max((6,decimal)))
        
        return self.b,self.pc

The above codes construct a 'class' for the serial production line.


In [ ]:
"""Input simulation time length (minute)"""
T=16*60

In [20]:
"""Input cylce time (minunte)"""
cycle_time=[1/5,
            1/2,
            1/4,
            1/3]


In [ ]:
"""Input maximum buffer capacities (parts)"""
buffer_capacity=[100,
                 200,
                 200]


In [ ]:
"""Input initial buffer levels (parts)"""
buffer_initial=[50,
                100,
                100]


In [ ]:
"""Input opportunity windows"""
downtime=([],
          [],
          [],
          [])


For example, to input the following opportunity windows
    machine 1: stops at t=10min and lasts for 25min, stops at t=80min and lasts for 20min
    machine 2: no opportunity window
    machine 3: stops at t=50min and lasts for 10min
    machine 4: no opportunity window

Then, you can input the opportunity window as followings:
<br>    downtime=([[10,25],[80,20]],
<br>              [],
<br>              [[50,10]],
<br>              [])

You may follow the format of the example to input your own opportunity windows. Be careful with the usage of square barackts.

In [21]:
line2=serial_line(cycle_time,buffer_capacity,buffer_initial)     #construct serial production line with the given parameters
b,pc=line2.run(T,downtime,decimal=2)                             #run the serial production line and get results

In [22]:
print("\nProduction Count of Machine 1 =",pc.M1.iloc[-1])             #index -1 points to the last element
print("Production Count of Machine 2 =",pc.M2.iloc[-1])
print("Production Count of Machine 3 =",pc.M3.iloc[-1])
print("Production Count of Machine 4 =",pc.M4.iloc[-1],"\n")


Production Count of Machine 1 = 1970.0
Production Count of Machine 2 = 1920.0
Production Count of Machine 3 = 2020.0
Production Count of Machine 4 = 2120.0 



In [23]:
print("\nBuffer Level of Buffer 2 at t = 50 mins =",b.B2[50])
print("Buffer Level of Buffer 2 at t = 50 mins =",b.B3[50])
print("Buffer Level of Buffer 2 at t = 50 mins =",b.B4[50],"\n")


Buffer Level of Buffer 2 at t = 50 mins = 100.0
Buffer Level of Buffer 2 at t = 50 mins = 0.0
Buffer Level of Buffer 2 at t = 50 mins = 150.0 



You may want to add some more lines down here to plot the chart you want.
You may refer to the codes in part 1 when making your own chart.

In [24]:
#add you code here

